# Create TotalModuleQty Column

This notebook describes the process in creating and adding the TotalModuleQty column to main dataset from Berkeley Lab's *Tracking the Sun* report. This column contains the sum of the total number of modules for each project listed in the dataset.

## Relevant Packages

The following package was imported for manipulating the dataset:

In [1]:
import pandas as pd

## Dataset

Next, the dataset (.csv) was imported.

In [2]:
# Main dataset
data = pd.read_csv('datasets/TTS_data.csv')

/home/tessmachado/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2,3,13,28,29,30,31,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Combine Datasets

These datasets are pared down from the original two datasets and only contain the following columns: 'moduleQty_1',	'moduleQty_2',	'moduleQty_3', 'mod_efficiency1'
- Note that the previous imported dataset also combines the datasets provided by Berkely Lab. These new datasets with only a couple columns were created to also demonstrate how we combined the datasets.

In [16]:
# Parsed down dataset names:
d1 = 'datasets/part1.csv'
d2 = 'datasets/part2.csv'

The datasets were combined by utilizing the same column names.

In [17]:
# Combine data (column names match)
comb_df = pd.concat(map(pd.read_csv, [d1,d2]), ignore_index = True, sort = False)

Remove rows with no efficiency data:
- The main dataset only contains the data where the efficiency ('mod_efficiency1') has reported data. If there is no reported data, this was indicated with a -1. The 'mod_efficiency1' column in the two parsed datasets were needed to attain the same shape as the primary dataset by dropping rows with -1.

In [18]:
# Drop rows where there is no data in the efficiency column
comb_df = comb_df.where(comb_df['mod_efficiency1'] != -1).dropna()

Reset index: 
- Due to dropping rows after the datasets were combined, the index was reset.

In [19]:
# Resets index to account for dropped rows
comb_df.reset_index(drop = True, inplace = True)

Drop efficiency column since not needed for calculation

In [21]:
# Drops 'mod_efficiency1' column
comb_df.drop('mod_efficiency1', axis=1, inplace=True)

In [22]:
comb_df.head()

,moduleQty_1,moduleQty_2,moduleQty_3
0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0


## Row Calculation

In order to calculate the total number of modules, the sum of the 3 columns were calculated while also ignoring the cells without data (indicated with -1). 

In [23]:
# Counts -1 values in a row
def count_na(j):
    count = 0
    for k in j:
        if k == -1:
            count += 1
    return count

# calculates the total: sum of the number of modules in each row
def find_sum(j):
    #print(j)
    sum_qty = 0
    for k in j:
        if k == -1: # ignores column if contains -1
            continue
        else:
            sum_qty += k
    return sum_qty

# initialize column in parsed dataset
comb_df['TotalModules'] = comb_df['moduleQty_1'] + comb_df['moduleQty_2']

# calculates sum of modules for each row
for i, j in comb_df.iterrows(): # iterates through each row in parsed dataset
    if count_na(j) == 3: # if there are three -1 values, assigns -1 in row
        comb_df['TotalModules'][i] = -1
    else:
        comb_df['TotalModules'][i] = find_sum(j) # calls function: finds sum


## Appends new column to main dataset

In [25]:
data['TotalModuleQty'] = comb_df['TotalModules']

## Save dataset with new column as .csv

In [29]:
data.to_csv('TTS_data.csv')